In [40]:
import requests
import pandas as pd
import json
import numpy as np

In [41]:
def read_mutual_fund_categories_amfi():
    df_mf_list = pd.read_csv('https://portal.amfiindia.com/DownloadSchemeData_Po.aspx?mf=0', header = 'infer')
    df_mf_list.columns = df_mf_list.columns.str.strip()
    df_mf_list.replace(np.nan,'', inplace = True)
    scheme_categories = df_mf_list['Scheme Category'].unique() ### use df_mf_list_1 after remove the category condition to get this
    df_mf_list_1 = df_mf_list[(df_mf_list['Closure Date'] == '') & (df_mf_list['Scheme Category'] == 'Equity Scheme - Flexi Cap Fund') & (df_mf_list['Scheme NAV Name'].str.contains("Direct") == True) & (df_mf_list['Scheme NAV Name'].str.contains("Growth") == True)].reset_index()
    return df_mf_list_1

def get_mf_price_data_mfapi(code, ):
    mf_data_response = requests.get(f"https://api.mfapi.in/mf/{code}", allow_redirects = True)
    mf_data_1 = json.loads(mf_data_response.content.decode('utf-8'))
    # mf_data = pd.json_normalize(mf_data_1['data'])
    mf_data = pd.json_normalize(mf_data_1)
    mf_data_df = pd.DataFrame.from_dict(mf_data['data'][0])
    mf_data_df['date'] = pd.to_datetime(mf_data_df['date'], format = '%d-%m-%Y')
    return mf_data_df

In [24]:
def read_mutual_fund_list_mfapi():
    response = requests.get("https://api.mfapi.in/mf", allow_redirects = True)
    data = json.loads(response.content.decode('utf-8'))
    df_mf_list = pd.json_normalize(data)

In [45]:
if __name__ == "__main__":
    all_funds = []
    mutual_fund_list = read_mutual_fund_categories_amfi()
    for index, row in mutual_fund_list.iterrows():
        code = row['Code']
        mf_data_df = get_mf_price_data_mfapi(code)
        mf_data_df['Scheme Code'] = row['Code']
        mf_data_df['Scheme Name'] = row['Scheme Name']
        mf_data_df['Scheme Category'] = row['Scheme Category']
        all_funds.append(mf_data_df)
        print(f"{code} - {row['Scheme Name']}")
final_df = pd.concat(all_funds, ignore_index=True)
final_df = final_df[['Scheme Code', 'Scheme Name', 'Scheme Category', 'date', 'nav']]
final_df.to_csv("mf_data.csv")

118883 - 118883 - Taurus Flexi Cap Fund
118535 - 118535 - Franklin India Flexi Cap Fund
120662 - 120662 - UTI - Flexi Cap Fund.
120046 - 120046 - HSBC Flexi Cap Fund
120564 - 120564 - Aditya Birla Sun Life Flexi Cap Fund
118424 - 118424 - BANDHAN Flexi Cap Fund
119292 - 119292 - Union Flexi Cap Fund
129046 - 129046 - Motilal Oswal Flexi Cap Fund
140353 - 140353 - Edelweiss Flexi Cap Fund
144546 - 144546 - Tata Flexi Cap Fund
144905 - 144905 - Shriram Flexi Cap Fund
148990 - 148990 - ICICI Prudential Flexicap fund-
149094 - 149094 - Nippon India Flexi Cap Fund
149104 - 149104 - Mahindra Manulife Flexi Cap Fund
149450 - 149450 - Samco Flexi Cap Fund
150387 - 150387 - Baroda BNP Paribas Flexi Cap Fund
150571 - 150571 - Sundaram Flexicap Fund
151379 - 151379 - ITI Flexi Cap Fund
151412 - 151412 - Mirae Asset Flexi Cap Fund
151895 - 151895 - Bajaj Finserv Flexi Cap Fund
151917 - 151917 - NJ Flexi Cap Fund
152135 - 152135 - Helios Flexi Cap Fund
152584 - 152584 - TRUSTMF Flexi Cap Fund
